In [1]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.0f}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

## Average Miles Walking and Biking per Day by Resident

In [3]:
# trip data
trip = pd.read_csv(output_path / 'agg\dash\person_trips.csv')
# person data
person = pd.read_csv(output_path / 'agg\dash\person_geog.csv')
# vmt data
vmt = pd.read_csv(output_path / 'agg/dash/person_vmt.csv')

# list of equity geographies
equity_geogs = summary_config['hh_equity_geogs']
# not_equity_geogs = ["NOT in " + item for item in equity_geogs]
# TRIPS
df_trip = trip.copy()
# add home RGC
df_trip['is_rgc'] = 'Not in RGC'
df_trip.loc[df_trip['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'
# add trip type
df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'

# PERSONS
df_person = person.copy()
# add home RGC
df_person['is_rgc'] = 'Not in RGC'
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

# VMT
df_vmt = vmt.copy()
# add home RGC
df_vmt['is_rgc'] = 'Not in RGC'
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

# Select only walk and bike trips
df_vmt_bp = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])].copy()
# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)].copy()

# # not in equity geography
# df_person[not_equity_geogs] = 1 - df_person[equity_geogs]
# df_trip[not_equity_geogs] = 1 - df_trip[equity_geogs]
# df_vmt[not_equity_geogs] = 1 - df_vmt[equity_geogs]
# df_vmt_bp[not_equity_geogs] = 1 - df_vmt_bp[equity_geogs]

# total population by equity geography
equity_geogs_population = df_person[equity_geogs].apply(lambda x: x * df_person['psexpfac']).sum().reset_index()
equity_geogs_population.columns = ['Equity Group', 'psexpfac']

# total population by "NOT in" equity geography
# not_equity_geogs_population = df_person[not_equity_geogs].apply(lambda x: x * df_person['psexpfac']).sum().reset_index()
# not_equity_geogs_population.columns = ['Equity Group', 'psexpfac']
df_vmt_bp2 = df_vmt_bp.copy()
df_vmt_bp2['mode'] = 'Walk and Bike'
# add walk and bike
df_vmt_bp2 = pd.concat([df_vmt_bp, df_vmt_bp2])

def walk_bike_per_person(geog, map=False):
    
    # miles by mode and geography
    df1 = df_vmt_bp2.groupby([geog, 'mode'], as_index=False)['travdist_wt'].sum().set_index(geog)

    if map:
        df1.index = df1.index.astype('int').map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

    # add total miles in region
    df1_region = df_vmt_bp2.groupby(['mode'])['travdist_wt'].sum().reset_index()
    df1_region[geog] = 'Region'
    df1_region = df1_region.set_index(geog)
    df1 = pd.concat([df1, df1_region])

    # population by geography
    df3 = df_person.groupby(geog, as_index=False)['psexpfac'].sum().set_index(geog)
    if map:
        df3.index = df3.index.astype('int').map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })
    df3.loc['Region',:] = df3.sum(axis=0)

    # calculate average miles per person
    df = df1.merge(df3, on=geog)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']

    return df.pivot(columns='mode', values='Average Miles per Person')

In [4]:
pd.set_option('display.float_format', '{:,.2f}'.format)
walk_bike_per_person('hh_county')

mode,Bike,Walk,Walk and Bike
hh_county,,,
King,0.21,0.58,0.79
Kitsap,0.14,0.52,0.65
Pierce,0.17,0.48,0.65
Region,0.19,0.53,0.72
Snohomish,0.16,0.47,0.63


In [5]:
walk_bike_per_person('is_rgc')

mode,Bike,Walk,Walk and Bike
is_rgc,,,
In RGC,0.26,1.05,1.31
Not in RGC,0.18,0.50,0.68
Region,0.19,0.53,0.72


In [6]:
walk_bike_per_person('hh_rgc')

mode,Bike,Walk,Walk and Bike
hh_rgc,,,
Auburn,0.37,0.63,1.00
Bellevue,0.26,0.87,1.12
Bothell Canyon Park,0.21,0.57,0.78
Bremerton,0.18,0.94,1.13
Burien,0.32,0.57,0.89
Everett,0.16,0.76,0.92
Federal Way,0.22,0.64,0.85
Greater Downtown Kirkland,0.24,0.62,0.86
Kent,0.36,0.67,1.04


In [7]:
walk_bike_per_person('hh_rg_proposed')

mode,Bike,Walk,Walk and Bike
hh_rg_proposed,,,
Cities and Towns,0.14,0.44,0.58
Core Cities,0.20,0.50,0.70
High Capacity Transit Communities,0.18,0.48,0.66
Metropolitan Cities,0.22,0.73,0.95
Region,0.19,0.53,0.72
Rural Areas,0.16,0.32,0.48
Urban Unincorporated Areas,0.15,0.44,0.59


In [8]:
df = pd.DataFrame()
for label, col in {
    "People of Color": "hh_efa_poc",
    "Income": "hh_efa_pov200",
    "LEP": "hh_efa_lep",
    "Disability": "hh_efa_dis",
    "Older Adults": "hh_efa_older",
    "Youth": "hh_efa_youth"
    }.items():
    _df = walk_bike_per_person(col, map=True)
    _df = _df.drop('Region', axis=0)
    _df['Group'] = label
    df = pd.concat([df, _df])
df = df.reset_index()
df.rename(columns={'index': 'EFA Type'}, inplace=True)
df
# df[['Group', 'EFA Type', 'Walk', 'Bike', 'Walk and Bike']] 

mode,EFA Type,Bike,Walk,Walk and Bike,Group
0,Above Regional Average,0.20,0.57,0.77,People of Color
1,Below Regional Average,0.18,0.51,0.68,People of Color
2,Higher Share of Equity Population,0.21,0.56,0.77,People of Color
3,Above Regional Average,0.19,0.55,0.74,Income
4,Below Regional Average,0.18,0.51,0.69,Income
5,Higher Share of Equity Population,0.22,0.61,0.82,Income
6,Above Regional Average,0.20,0.53,0.72,LEP
7,Below Regional Average,0.18,0.53,0.71,LEP
8,Higher Share of Equity Population,0.21,0.54,0.75,LEP
9,Above Regional Average,0.19,0.53,0.72,Disability


## Median Trip Distance by Mode and Home Location 

In [9]:
pd.set_option('display.float_format', '{:,.1f}'.format)
df = pd.read_csv(Path(output_path, 'agg/dash/trip_median_county.csv'))
df = df[df['hh_county']!='Outside Region']
df = df[df['mode'].isin(['Bike','Walk','SOV','HOV2','Transit'])]
df = pd.pivot_table(df, index='hh_county', columns='mode', aggfunc='first', values='travdist')
df.rename(columns={'SOV': 'Drive Alone',
                   'HOV2': 'Shared Ride'}, inplace=True)
df[['Drive Alone','Shared Ride','Transit','Bike','Walk']]

mode,Drive Alone,Shared Ride,Transit,Bike,Walk
hh_county,,,,,
King,3.6,2.7,4.0,1.7,0.7
Kitsap,3.1,2.5,11.7,1.7,0.7
Pierce,3.8,2.8,3.9,1.8,0.8
Snohomish,3.8,2.7,4.0,1.8,0.8


In [10]:
df = pd.read_csv(Path(output_path, 'agg/dash/trip_median_rg.csv'))
df = df[df['hh_rg_proposed']!='Outside Region']
df = df[df['mode'].isin(['Bike','Walk','SOV','HOV2','Transit'])]
df = pd.pivot_table(df, index='hh_rg_proposed', columns='mode', aggfunc='first', values='travdist')
df.rename(columns={'SOV': 'Drive Alone',
                   'HOV2': 'Shared Ride'}, inplace=True)
df[['Drive Alone','Shared Ride','Transit','Bike','Walk']]

mode,Drive Alone,Shared Ride,Transit,Bike,Walk
hh_rg_proposed,,,,,
Cities and Towns,4.4,2.7,8.0,1.6,0.8
Core Cities,3.5,2.6,6.3,1.9,0.8
High Capacity Transit Communities,3.8,2.7,7.6,1.9,0.9
Metropolitan Cities,2.5,2.0,3.2,1.5,0.6
Rural Areas,6.3,5.3,13.3,3.0,1.0
Urban Unincorporated Areas,4.2,3.1,9.2,2.0,1.0


In [11]:
df = pd.read_csv(Path(output_path, 'agg/dash/trip_median_rgc.csv'))
df = df[df['hh_rgc']!='Outside Region']
df = df[df['mode'].isin(['Bike','Walk','SOV','HOV2','Transit'])]
df = pd.pivot_table(df, index='hh_rgc', columns='mode', aggfunc='first', values='travdist')
df.rename(columns={'SOV': 'Drive Alone',
                   'HOV2': 'Shared Ride'}, inplace=True)
df[['Drive Alone','Shared Ride','Transit','Bike','Walk']]

mode,Drive Alone,Shared Ride,Transit,Bike,Walk
hh_rgc,,,,,
Auburn,2.2,1.6,10.6,1.9,0.4
Bellevue,1.4,1.0,4.8,0.8,0.4
Bothell Canyon Park,3.5,2.4,4.5,2.5,0.8
Bremerton,1.5,1.2,24.6,1.2,0.4
Burien,4.2,2.7,8.2,1.4,0.4
Everett,1.2,0.9,1.2,0.8,0.4
Federal Way,2.5,1.9,4.6,1.7,0.5
Greater Downtown Kirkland,3.2,2.1,5.3,1.7,0.5
Kent,2.5,1.6,6.3,2.4,0.4


## Median Time Spent Walking/Biking
Time in minutes

In [12]:
pd.set_option('display.float_format', '{:,.1f}'.format)
df = pd.read_csv(Path(output_path, 'agg/dash/trip_median_county_time.csv'))
df = df[df['hh_county']!='Outside Region']
df = df[df['mode'].isin(['Bike','Walk','SOV','HOV2','Transit'])]
df = pd.pivot_table(df, index='hh_county', columns='mode', aggfunc='first', values='travtime')
df.rename(columns={'SOV': 'Drive Alone',
                   'HOV2': 'Shared Ride'}, inplace=True)
df[['Drive Alone','Shared Ride','Transit','Bike','Walk']]

mode,Drive Alone,Shared Ride,Transit,Bike,Walk
hh_county,,,,,
King,12.3,10.4,24.1,11.2,13.1
Kitsap,10.7,9.5,68.6,11.1,14.5
Pierce,12.2,10.5,35.8,12.3,16.3
Snohomish,11.7,9.4,31.1,12.1,16.5


In [13]:
df = pd.read_csv(Path(output_path, 'agg/dash/trip_median_rg_time.csv'))
df = df[df['hh_rg_proposed']!='Outside Region']
df = df[df['mode'].isin(['Bike','Walk','SOV','HOV2','Transit'])]
df = pd.pivot_table(df, index='hh_rg_proposed', columns='mode', aggfunc='first', values='travtime')
df.rename(columns={'SOV': 'Drive Alone',
                   'HOV2': 'Shared Ride'}, inplace=True)
df[['Drive Alone','Shared Ride','Transit','Bike','Walk']]

mode,Drive Alone,Shared Ride,Transit,Bike,Walk
hh_rg_proposed,,,,,
Cities and Towns,12.2,9.4,45.0,10.4,15.2
Core Cities,11.2,9.5,38.6,12.5,16.3
High Capacity Transit Communities,11.7,9.7,39.8,12.8,17.2
Metropolitan Cities,11.5,9.9,20.5,9.8,12.0
Rural Areas,15.7,14.2,59.9,20.0,19.1
Urban Unincorporated Areas,12.3,10.3,47.7,13.2,20.2


In [14]:
df = pd.read_csv(Path(output_path, 'agg/dash/trip_median_rgc_time.csv'))
df = df[df['hh_rgc']!='Outside Region']
df = df[df['mode'].isin(['Bike','Walk','SOV','HOV2','Transit'])]
df = pd.pivot_table(df, index='hh_rgc', columns='mode', aggfunc='first', values='travtime')
df.rename(columns={'SOV': 'Drive Alone',
                   'HOV2': 'Shared Ride'}, inplace=True)
df[['Drive Alone','Shared Ride','Transit','Bike','Walk']]

mode,Drive Alone,Shared Ride,Transit,Bike,Walk
hh_rgc,,,,,
Auburn,9.6,8.4,47.0,13.0,8.5
Bellevue,8.2,7.1,26.2,5.4,7.1
Bothell Canyon Park,11.9,9.6,38.2,16.4,15.3
Bremerton,9.9,9.5,58.2,8.2,9.0
Burien,11.6,9.0,41.4,9.4,8.1
Everett,7.6,6.7,13.9,5.7,8.1
Federal Way,9.1,7.6,30.3,11.2,10.0
Greater Downtown Kirkland,10.3,8.5,32.5,11.0,10.0
Kent,8.9,7.0,36.8,16.1,8.0


## % Population Walking or Biking for Transportation
Does not include people making exercise trips

In [15]:
trip = pl.read_csv(r'../../../../outputs/daysim/_trip.tsv',separator='\t')
person = pl.read_csv(r'../../../../outputs/daysim/_person.tsv',separator='\t')
hh = pl.read_csv(r'../../../../outputs/daysim/_household.tsv',separator='\t')

trip_person = trip.join(person, on=["hhno", "pno"], how="left")
bike_walk_trips = trip_person.filter(pl.col("mode").is_in([1, 2]))

# Get unique persons with at least one bike/walk trip
bike_walk_persons = bike_walk_trips.select(["hhno", "pno"]).unique()
bike_walk_persons = bike_walk_persons.with_columns(bike_walk=pl.lit(1))

# Join back to all persons, mark bike_walk as False if not present
person_with_bike_walk = person.join(bike_walk_persons, on=["hhno", "pno"], how="left")
person_with_bike_walk = person_with_bike_walk.join(hh, on='hhno', how='left')


In [16]:
pd.set_option('display.float_format', '{:.1%}'.format)
list_cols = ['ParcelID','CountyName','GrowthCenterName','rg_proposed']
async_engine = create_engine('sqlite:///' + summary_config['sc_run_path'] + '/inputs/db/' + config['db_name'])
parcel_geog = pl.read_database(
    query= f"SELECT {', '.join(list_cols)} FROM " + "parcel_" + config["base_year"] + "_geography",
    connection=async_engine.connect()
)

# person_with_bike_walk.merge(parcel_geog, how='left', left_on='hhparcel', right_on='ParcelID')
person_with_bike_walk = person_with_bike_walk.join(parcel_geog, left_on='hhno', right_on='ParcelID', how='left')

person_with_bike_walk = person_with_bike_walk.to_pandas()

person_with_bike_walk['bike_walk'] = person_with_bike_walk['bike_walk'].fillna(0)
df = pd.pivot_table(person_with_bike_walk, index='CountyName', columns='bike_walk', aggfunc='sum', values='psexpfac')
df['% Biking or Walking'] = df[1]/(df[0]+df[1])
df[['% Biking or Walking']]

bike_walk,% Biking or Walking
CountyName,
King,30.8%
Kitsap,30.7%
Outside Region,25.8%
Pierce,23.9%
Snohomish,20.4%


In [17]:
df = pd.pivot_table(person_with_bike_walk, index='rg_proposed', columns='bike_walk', aggfunc='sum', values='psexpfac')
df['% Biking or Walking'] = df[1]/(df[0]+df[1])
df[['% Biking or Walking']]

bike_walk,% Biking or Walking
rg_proposed,
Cities and Towns,27.5%
Core Cities,29.2%
High Capacity Transit Communities,24.2%
Metropolitan Cities,28.6%
Rural Areas,25.2%
Urban Unincorporated Areas,25.4%


In [18]:
df = pd.pivot_table(person_with_bike_walk, index='GrowthCenterName', columns='bike_walk', aggfunc='sum', values='psexpfac')
df['% Biking or Walking'] = df[1]/(df[0]+df[1])
df[['% Biking or Walking']]

bike_walk,% Biking or Walking
GrowthCenterName,
Auburn,32.0%
Bellevue,27.2%
Bothell Canyon Park,20.3%
Bremerton,33.5%
Burien,21.8%
Everett,20.4%
Federal Way,28.7%
Greater Downtown Kirkland,20.2%
Issaquah,21.4%


## Bike and Walk Tours by Purpose
Tours where most or all trips are by the mode.

In [19]:
pd.set_option('display.float_format', '{:.1%}'.format)
df = pd.read_csv(Path(output_path, 'agg/dash/tour_total.csv'))

df['Tour Mode'] = df['tmodetp'].map({'SOV': 'Drove Alone',
                                     'HOV2': 'Shared Ride',
                                     'HOV3+': 'Shared Ride',
                                     'Transit': 'Transit',
                                     'Walk': 'Walk',
                                     'Bike': 'Bike'
                                     })

df = pd.pivot_table(df, index='pdpurp', columns='Tour Mode', values='toexpfac', aggfunc='sum')
df = df/df.sum()
df[['Transit', 'Walk', 'Bike','Drove Alone', 'Shared Ride', ]]

Tour Mode,Transit,Walk,Bike,Drove Alone,Shared Ride
pdpurp,,,,,
Escort,0.0%,7.1%,2.8%,1.0%,22.8%
Meal,8.6%,10.2%,1.7%,4.9%,7.5%
Personal Business,12.7%,5.6%,4.1%,8.3%,5.8%
School,20.1%,4.1%,16.1%,4.2%,11.7%
Shop,16.6%,12.0%,7.7%,15.4%,10.9%
Social,18.5%,52.1%,39.2%,20.9%,22.6%
Work,23.5%,9.1%,28.4%,45.4%,18.6%


In [20]:
transit_jobs_access = pd.read_csv(output_path / 'access/transit_jobs_access.csv', 
                                  usecols=['geography', 'value', 'geography_group'])
walk_bike_jobs_access = pd.read_csv(output_path / 'access/walk_bike_jobs_access.csv', 
                                  usecols=['geography_value', 'jobs_1_mile_walk', 'jobs_3_mile_bike', 'geography_group']).\
                                  rename(columns={'geography_value': 'geography'})

parcel_emp = pl.read_csv(output_path / 'landuse/parcels_urbansim.txt', 
                         separator=' ', 
                         columns=['parcelid','emptot_p']).to_pandas()


# jobs access in equity geographies
equity_geogs = summary_config['equity_geogs']

In [21]:
def process_access_data(jobs_access):
    df_access = jobs_access.copy()
    # rename region
    df_access.loc[jobs_access['geography_group'] == 'region', 'geography'] = 'Region'
    # rename rgc
    df_access.loc[jobs_access['geography_group'] == 'rgc_binary', 'geography'] = ['Not in RGC', 'In RGC']

    df_access_equity = df_access.loc[
        df_access['geography_group'].isin(equity_geogs)].copy()
    
    df_access_equity['geography'] = df_access_equity['geography'].map({"0.0": 'Below Regional Average', 
                                                                           "1.0": 'Above Regional Average', 
                                                                           "2.0": 'Higher Share of Equity Population'}
                                                                           )

    return df_access, df_access_equity


df_access_t, df_access_equity_t = process_access_data(transit_jobs_access)
df_access_bp, df_access_equity_bp = process_access_data(walk_bike_jobs_access)
tot_jobs = parcel_emp['emptot_p'].sum()

In [22]:
def job_access_geog(access_table,geog):
    df = access_table.loc[access_table['geography_group'].isin([geog, 'region'])].\
        rename(columns={'value': 'Jobs within 45-minute Transit Commute'}).\
        drop(columns=['geography_group']).\
        set_index('geography')

    df['% Total Jobs'] = df['Jobs within 45-minute Transit Commute'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')

    return df


## Average Jobs Accessible within 1 Mile Walk and 3 Mile Bike
Note that this is not using the bike network, but is instead using the all-streets network.

Average accessible jobs are weighted averages based on parcel household population.

In [23]:
def bp_job_access_geog(access_table,geog):
    df = access_table.loc[access_table['geography_group'].isin(['region', geog])].\
        rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}).\
        drop(columns=['geography_group']).\
        set_index('geography')

    df['% Total Jobs (1-mile walk)'] = df['Jobs within 1-mile Walk'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')
    df['% Total Jobs (3-mile bike)'] = df['Jobs within 3-mile Bike'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')

    return df

In [24]:
df = bp_job_access_geog(df_access_bp,'CountyName')
df = df[df.index != 'Outside Region']
df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
King,1851581.4%,8491318.2%,0.9%,3.9%
Kitsap,123603.8%,801435.8%,0.1%,0.4%
Pierce,250799.4%,1816317.8%,0.1%,0.8%
Snohomish,200577.3%,1770284.3%,0.1%,0.8%
Region,1116391.8%,5425384.5%,0.5%,2.5%


In [25]:
df_rgc = bp_job_access_geog(df_access_bp,'rgc_binary')
df = bp_job_access_geog(df_access_bp,'GrowthCenterName')

pd.concat([df_rgc, df.loc[~df.index.isin(['Region','Not in RGC'])]], axis=0)

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
Region,1116391.8%,5425384.5%,0.5%,2.5%
Not in RGC,291218.7%,3486837.5%,0.1%,1.6%
In RGC,8887771.9%,23682384.8%,4.1%,11.0%
Auburn,1054099.2%,4032406.6%,0.5%,1.9%
Bellevue,5918796.9%,11069945.5%,2.7%,5.1%
Bothell Canyon Park,853850.3%,2174825.2%,0.4%,1.0%
Bremerton,1153580.2%,3438727.8%,0.5%,1.6%
Burien,482884.4%,1339977.6%,0.2%,0.6%
Everett,1590767.7%,3972210.7%,0.7%,1.8%


In [26]:
bp_job_access_geog(df_access_bp,'rg_proposed')

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
Region,1116391.8%,5425384.5%,0.5%,2.5%
Cities and Towns,98372.0%,894778.4%,0.0%,0.4%
Core Cities,341480.7%,2858704.8%,0.2%,1.3%
High Capacity Transit Communities,142765.4%,1523749.1%,0.1%,0.7%
Metropolitan Cities,2943242.3%,12807447.8%,1.4%,5.9%
Rural Areas,14229.7%,238706.5%,0.0%,0.1%
Urban Unincorporated Areas,44260.7%,681561.8%,0.0%,0.3%


In [27]:
df = pd.DataFrame()
for label, col in {
    "People of Color": "equity_focus_areas_2023__efa_poc",
    "Income": "equity_focus_areas_2023__efa_pov200",
    "LEP": "equity_focus_areas_2023__efa_lep",
    "Disability": "equity_focus_areas_2023__efa_dis",
    "Older Adults": "equity_focus_areas_2023__efa_older",
    "Youth": "equity_focus_areas_2023__efa_youth"
    }.items():
    _df = bp_job_access_geog(df_access_equity_bp, col)
    _df['Group'] = label
    df = pd.concat([df, _df])
df = df.reset_index()
df.rename(columns={'geography': 'EFA Type'}, inplace=True)
df[['Group', 'EFA Type', 'Jobs within 1-mile Walk',	'Jobs within 3-mile Bike',	'% Total Jobs (1-mile walk)','% Total Jobs (3-mile bike)']]

,Group,EFA Type,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
0,People of Color,Below Regional Average,537412.1%,4193526.2%,0.2%,1.9%
1,People of Color,Above Regional Average,1850947.0%,7173055.8%,0.9%,3.3%
2,People of Color,Higher Share of Equity Population,1722871.8%,6399390.5%,0.8%,3.0%
3,Income,Below Regional Average,939442.9%,5241950.5%,0.4%,2.4%
4,Income,Above Regional Average,1389941.1%,5625848.9%,0.6%,2.6%
5,Income,Higher Share of Equity Population,1365137.5%,5832021.5%,0.6%,2.7%
6,LEP,Below Regional Average,1136842.3%,5781837.4%,0.5%,2.7%
7,LEP,Above Regional Average,1328380.1%,5184210.9%,0.6%,2.4%
8,LEP,Higher Share of Equity Population,774536.1%,4367359.0%,0.4%,2.0%
9,Disability,Below Regional Average,965078.3%,5687011.1%,0.4%,2.6%
